## Part3
Using the models.ldamodel module from the gensim library, run topic modeling over the corpus. Explore different numbers of topics (varying from 5 to 50), and settle for the parameter which returns topics that you consider to be meaningful at first sight.

In [1]:
# imports
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from gensim import corpora, models, utils
from nltk.stem import WordNetLemmatizer


/home/christophe/Application/anaconda3/lib/python3.5/site-packages/gensim/utils.py:1015: UserWarning: Pattern library is not installed, lemmatization won't be available.
  warnings.warn("Pattern library is not installed, lemmatization won't be available.")


In [2]:
data = pd.read_csv('hillary-clinton-emails/Emails.csv', index_col=0).dropna()
texts = pd.concat((data.ExtractedBodyText ,data.ExtractedSubject), axis=1)

In [3]:
sw = ['re', 'fw', 'fvv', 'fwd']

To improve the result of the lda model, we group the mails by Subject. We filter the subjects to remove the keywords 're', 'fw', 'fvv', 'fwd'

In [4]:
def filt(row):
    t = utils.simple_preprocess(row.ExtractedSubject)
    filt = list(filter(lambda x: x not in sw, t))
    return ' '.join(filt)
texts['ExtractedSubject'] = texts.apply(filt, axis=1)
texts = texts.groupby(by='ExtractedSubject', as_index=False).apply(lambda x: (x + ' ').sum())

In [5]:
texts.head()

,ExtractedBodyText,ExtractedSubject
0,- Benghazi\nSounds good. Thx--be sure I see hi...,
1,"Jake, when you can, I think I speak for the gr...",aar
2,Not until I have a report from Mitchell. No on...,abu mazen abu mazen abu mazen abu mazen abu ma...
3,I just wanted to share how much the Secretary ...,aceh paper
4,See below - confidential.\nAlso for WJC.,acting srsg mulet


we concat the e-mail body wit the subject.

In [6]:
texts.ExtractedBodyText.fillna('',inplace=True)
texts.ExtractedSubject.fillna('',inplace=True)
texts['SubjectBody'] = texts.ExtractedBodyText +  ' ' + texts.ExtractedSubject
mails = texts.SubjectBody

to get more meaningfull topics, we filter out english stop words and some custom words that don't have much meaning as topics.

In [7]:
documents = []

custom = ['like', 'think', 'know', 'want', 'sure', 'thing', 'send', 'sent', 'speech', 'print', 'time','want', 'said', 'maybe', 'today', 'tomorrow', 'thank', 'thanks']
english_stop_words = ["a", "about", "above", "above", "across", "after", "afterwards", "again", "against", "all", "almost", "alone", "along", "already", "also","although","always","am","among", "amongst", "amoungst", "amount",  "an", "and", "another", "any","anyhow","anyone","anything","anyway", "anywhere", "are", "around", "as",  "at", "back","be","became", "because","become","becomes", "becoming", "been", "before", "beforehand", "behind", "being", "below", "beside", "besides", "between", "beyond", "bill", "both", "bottom","but", "by", "call", "can", "cannot", "cant", "co", "con", "could", "couldnt", "cry", "de", "describe", "detail", "do", "done", "down", "due", "during", "each", "eg", "eight", "either", "eleven","else", "elsewhere", "empty", "enough", "etc", "even", "ever", "every", "everyone", "everything", "everywhere", "except", "few", "fifteen", "fify", "fill", "find", "fire", "first", "five", "for", "former", "formerly", "forty", "found", "four", "from", "front", "full", "further", "get", "give", "go", "had", "has", "hasnt", "have", "he", "hence", "her", "here", "hereafter", "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his", "how", "however", "hundred", "ie", "if", "in", "inc", "indeed", "interest", "into", "is", "it", "its", "itself", "keep", "last", "latter", "latterly", "least", "less", "ltd", "made", "many", "may", "me", "meanwhile", "might", "mill", "mine", "more", "moreover", "most", "mostly", "move", "much", "must", "my", "myself", "name", "namely", "neither", "never", "nevertheless", "next", "nine", "no", "nobody", "none", "noone", "nor", "not", "nothing", "now", "nowhere", "of", "off", "often", "on", "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our", "ours", "ourselves", "out", "over", "own","part", "per", "perhaps", "please", "put", "rather", "re", "same", "see", "seem", "seemed", "seeming", "seems", "serious", "several", "she", "should", "show", "side", "since", "sincere", "six", "sixty", "so", "some", "somehow", "someone", "something", "sometime", "sometimes", "somewhere", "still", "such", "system", "take", "ten", "than", "that", "the", "their", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "therefore", "therein", "thereupon", "these", "they", "thickv", "thin", "third", "this", "those", "though", "three", "through", "throughout", "thru", "thus", "to", "together", "too", "top", "toward", "towards", "twelve", "twenty", "two", "un", "under", "until", "up", "upon", "us", "very", "via", "was", "we", "well", "were", "what", "whatever", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", "wherein", "whereupon", "wherever", "whether", "which", "while", "whither", "who", "whoever", "whole", "whom", "whose", "why", "will", "with", "within", "without", "would", "yet", "you", "your", "yours", "yourself", "yourselves", "the"]
sw =stopwords.words('english') + sw + custom + english_stop_words
for text in mails:
    t = utils.simple_preprocess(text)
    filt = list(filter(lambda x: (x not in sw) and len(x) > 3, t))
    
    lemmatizer = WordNetLemmatizer()
    lemmatized = [lemmatizer.lemmatize(x) for x in filt]
    filt2 = list(filter(lambda x: (x not in sw) and len(x) > 3, lemmatized))
    documents.append(filt2)

To use the LDA model, we need to transform every document into a list of tuple (ID, term frequency).

In [8]:
dictionary = corpora.Dictionary(documents)
corpus = [dictionary.doc2bow(doc) for doc in documents]

Now we generate the LDA models with different numbers of topics.

In [9]:

import pprint
pp = pprint.PrettyPrinter(depth=2)
for i in range(5, 50, 10):
    print('------------------------------------')
    print(i, 'topics')
    lda = models.LdaModel(corpus, num_topics=i, id2word = dictionary)
    pp.pprint(lda.print_topics(lda.num_topics))
    print()

------------------------------------
5 topics
[(0,
  '0.008*"good" + 0.007*"issue" + 0.007*"schedule" + 0.007*"draft" + '
  '0.006*"state" + 0.005*"meeting" + 0.005*"secretary" + 0.005*"sheet" + '
  '0.005*"africa" + 0.005*"make"'),
 (1,
  '0.007*"email" + 0.007*"list" + 0.007*"secretary" + 0.006*"state" + '
  '0.006*"schedule" + 0.005*"make" + 0.005*"work" + 0.005*"statement" + '
  '0.004*"draft" + 0.004*"need"'),
 (2,
  '0.012*"press" + 0.011*"state" + 0.010*"strategic" + 0.010*"dialogue" + '
  '0.010*"office" + 0.010*"list" + 0.009*"clip" + 0.008*"secretary" + '
  '0.007*"talk" + 0.007*"schedule"'),
 (3,
  '0.009*"haiti" + 0.007*"good" + 0.007*"work" + 0.006*"foreign" + '
  '0.006*"meeting" + 0.005*"draft" + 0.005*"make" + 0.005*"help" + '
  '0.005*"right" + 0.005*"article"'),
 (4,
  '0.006*"work" + 0.006*"schedule" + 0.005*"cheryl" + 0.005*"discus" + '
  '0.005*"lunch" + 0.005*"state" + 0.005*"sheet" + 0.005*"issue" + '
  '0.005*"letter" + 0.004*"dinner"')]

-----------------------

when we choose the number of topics to be 25, the topics seem to be the most meaningfull.

In [10]:
lda = models.LdaModel(corpus, num_topics=25, id2word = dictionary)
pp.pprint(lda.print_topics(lda.num_topics))

[(0,
  '0.014*"family" + 0.012*"text" + 0.012*"davutoglu" + 0.011*"armenia" + '
  '0.011*"schedule" + 0.011*"turkey" + 0.010*"state" + 0.010*"work" + '
  '0.009*"table" + 0.009*"info"'),
 (1,
  '0.026*"dinner" + 0.018*"tech" + 0.014*"health" + 0.013*"abortion" + '
  '0.013*"maternal" + 0.009*"list" + 0.009*"love" + 0.009*"idea" + '
  '0.009*"copying" + 0.009*"romano"'),
 (2,
  '0.014*"copy" + 0.010*"check" + 0.009*"state" + 0.009*"benghazi" + '
  '0.009*"message" + 0.009*"make" + 0.008*"need" + 0.008*"hatfield" + '
  '0.008*"dalton" + 0.007*"good"'),
 (3,
  '0.017*"talk" + 0.014*"benghazi" + 0.010*"saudabayev" + 0.010*"mexico" + '
  '0.009*"clinton" + 0.007*"expectation" + 0.007*"update" + 0.007*"decision" + '
  '0.007*"honduras" + 0.007*"coup"'),
 (4,
  '0.014*"water" + 0.013*"state" + 0.011*"little" + 0.010*"billion" + '
  '0.010*"pakistan" + 0.010*"dollar" + 0.010*"goodwill" + 0.008*"work" + '
  '0.008*"schedule" + 0.008*"friday"'),
 (5,
  '0.015*"make" + 0.015*"email" + 0.009*"mash